# Viability tube for quadrotor

**Adapted from**: [YAP21, Section V.D] for the model defined in [B12], [M16, Section IV] and [M19, Section 6.1]

[B12] Bouffard, Patrick.
*On-board model predictive control of a quadrotor helicopter: Design, implementation, and experiments.*
CALIFORNIA UNIV BERKELEY DEPT OF COMPUTER SCIENCES, 2012.

[M16] Mitchell, Ian M., et al.
*Ensuring safety for sampled data systems: An efficient algorithm for filtering potentially unsafe input signals.*
2016 IEEE 55th Conference on Decision and Control (CDC). IEEE, 2016.

[M19] Mitchell, Ian M., Jacob Budzis, and Andriy Bolyachevets.
*Invariant, viability and discriminating kernel under-approximation via zonotope scaling.*
Proceedings of the 22nd ACM International Conference on Hybrid Systems: Computation and Control. 2019.

[YAP21] Yin, H., Arcak, M., Packard, A., & Seiler, P. (2021).
*Backward reachability for polynomial systems on a finite horizon.*
IEEE Transactions on Automatic Control, 66(12), 6025-6032.

In [1]:
using DynamicPolynomials
@polyvar x[1:6]
@polyvar u[1:2]
sinx5 = -0.166 * x[5]^3 + x[5]
cosx5 = -0.498 * x[5]^2 + 1
gn = 9.8
K = 0.89 / 1.4
d0 = 70
d1 = 17
n0 = 55
f = [
    x[3],
    x[4],
    0,
    -gn,
    x[6],
    -d0 * x[5] - d1 * x[6],
]
n_x = length(f)
g = [
    0         0
    0         0
    K * sinx5 0
    K * cosx5 0
    0         0
    0         n0
]
n_u = size(g, 2)

2

The constraints below are the same as [YAP21, M16, M19] except
[M16, M19] uses different bounds for `x[2]` and
[M16] uses different bounds for `x[5]`

In [2]:
using SumOfSquares
rectangle = [1.7, 0.85, 0.8, 1, π/12, π/2, 1.5, π/12]
X = BasicSemialgebraicSet(FullSpace(), typeof(x[1] + 1.0)[])
for i in eachindex(x)
    addinequality!(X, x[i] + rectangle[i]) # x[i] >= -rectangle[i]
    addinequality!(X, rectangle[i] - x[i]) # x[i] <= rectangle[i]
end
X

Basic semialgebraic Set defined by no equality
12 inequalities
 x[1] + 1.7 ≥ 0
 -x[1] + 1.7 ≥ 0
 x[2] + 0.85 ≥ 0
 -x[2] + 0.85 ≥ 0
 x[3] + 0.8 ≥ 0
 -x[3] + 0.8 ≥ 0
 x[4] + 1.0 ≥ 0
 -x[4] + 1.0 ≥ 0
 x[5] + 0.2617993877991494 ≥ 0
 -x[5] + 0.2617993877991494 ≥ 0
 x[6] + 1.5707963267948966 ≥ 0
 -x[6] + 1.5707963267948966 ≥ 0


The starting value for `k` is the following linear state-feedback
that maintains the quadrotor at the origin [YAP21, Remark 3].

In [3]:
using SparseArrays
x0 = zeros(n_x)
u0 = [gn/K, 0.0]

2-element Vector{Float64}:
 15.415730337078651
  0.0

The linearization of `f` is given by

In [4]:
x_dot = f + g * u
A = map(differentiate(x_dot, x)) do a
    a(x => x0, u => u0)
end

6×6 Matrix{Float64}:
 0.0  0.0  1.0  0.0    0.0    0.0
 0.0  0.0  0.0  1.0    0.0    0.0
 0.0  0.0  0.0  0.0    9.8    0.0
 0.0  0.0  0.0  0.0   -0.0    0.0
 0.0  0.0  0.0  0.0    0.0    1.0
 0.0  0.0  0.0  0.0  -70.0  -17.0

The linearization of `g` is given by:

In [5]:
B = map(differentiate(x_dot, u)) do b
    b(x => x0, u => u0)
end

6×2 Matrix{Float64}:
 0.0        0.0
 0.0        0.0
 0.0        0.0
 0.635714   0.0
 0.0        0.0
 0.0       55.0

We can compute the Linear-Quadratic Regulator using the same weight matrices
as [YAP21](https://github.com/heyinUCB/Backward-Reachability-Analysis-and-Control-Synthesis)

In [6]:
import MatrixEquations
S, v, K = MatrixEquations.arec(A, B, 10, 100)

([145.91648724513337 0.0 … 103.37066470647957 0.5749595745760094; 0.0 141.24000391586395 … -0.0 0.0; … ; 103.37066470647957 -0.0 … 559.7832728912887 2.949785658896485; 0.5749595745760094 0.0 … 2.949785658896485 0.5381411869254619], ComplexF64[-174.34978294240278 + 0.0im, -2.2185572278601233 + 2.2014118608231428im, -2.2185572278601233 - 2.2014118608231428im, -1.0008116468297334 + 0.0im, -1.4916433890176233 + 0.0im, -1.3477115902937555 + 0.0im], [0.0 3.1622776601686358 … 0.0 0.0; 3.162277660168052 0.0 … 16.223821123930666 2.9597765280900403], [1.611144845196192e-6 -0.008771626450488266 … 0.0 0.0; -0.0 0.0 … 0.011524906469290956 -0.0008272341629524921; … ; 0.0 0.0 … -0.04033717264252285 -0.04911829197708385; -2.499095953227872 -0.09890033030678862 … 0.0 0.0])

The corresponding quadratic regulator is:

In [7]:
P, _, _ = MatrixEquations.arec(A - B * K, 0.0, 10.0)

([14.546414232392811 -4.1519546509066404e-13 … 10.155770619177467 0.02874797872881651; -4.1519546509066404e-13 12.36303706137998 … -5.77308345064067e-13 -3.2019987351797165e-15; … ; 10.155770619177467 -5.77308345064067e-13 … 55.113616622956386 0.15534043417691945; 0.02874797872881651 -3.2019987351797165e-15 … 0.15534043417691945 0.028674598845285566], ComplexF64[-174.3497829424029 + 0.0im, -2.218557227859746 + 2.201411860823505im, -2.218557227859746 - 2.201411860823505im, -1.0008116468296286 + 0.0im, -1.4916433890159477 + 0.0im, -1.347711590295426 + 0.0im], [-1.8241315067739179e-6 0.02377516304458807 … 5.23307891122959e-13 9.079337655017754e-14; 2.0565674097552837e-17 -1.9400815594378327e-15 … 0.11166975002343552 -0.06136401345843784; … ; -0.15392719914799866 -0.959743422396598 … 5.337240729396093e-13 9.687042596588282e-14; 0.027421579759805473 -0.00707662916934221 … 8.557002917606841e-15 1.5699585489046389e-15])

This does not however take the constraints `X` into account.
To take the constraint into account,
we compute an ellipsoidal control invariant set using [LJ21, Corollary 9]
For this, we first compute the descriptor system described in [LJ21, Proposition 5].

[LJ21] Legat, Benoît, and Jungers, Raphaël M.
*Geometric control of algebraic systems.*
IFAC-PapersOnLine 54.5 (2021): 79-84.

In [8]:
nD = n_x + n_u
E = sparse(1:n_x, 1:n_x, ones(n_x), n_x, nD)
C = [A B]

6×8 Matrix{Float64}:
 0.0  0.0  1.0  0.0    0.0    0.0  0.0        0.0
 0.0  0.0  0.0  1.0    0.0    0.0  0.0        0.0
 0.0  0.0  0.0  0.0    9.8    0.0  0.0        0.0
 0.0  0.0  0.0  0.0   -0.0    0.0  0.635714   0.0
 0.0  0.0  0.0  0.0    0.0    1.0  0.0        0.0
 0.0  0.0  0.0  0.0  -70.0  -17.0  0.0       55.0

We know solve [LJ21, (13)]

In [9]:
using LinearAlgebra
import SCS
solver = optimizer_with_attributes(SCS.Optimizer, MOI.Silent() => true)
model = Model(solver)
@variable(model, Q[1:nD, 1:nD] in PSDCone())
cref = @constraint(model, Symmetric(-C * Q * E' - E * Q * C') in PSDCone())
@constraint(model, rect_ref[i in 1:nD], Q[i, i] <= rectangle[i])
@variable(model, volume)
q = [Q[i, j] for j in 1:nD for i in 1:j]
@constraint(model, [volume; q] in MOI.RootDetConeTriangle(nD))
@objective(model, Max, volume)
optimize!(model)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 6.10540e-01
  Dual objective value : 6.10520e-01

* Work counters
  Solve time (sec)   : 4.63324e-02


We now have the control-Lyapunov function `V(x, u) = [x, u]' inv(Q) [x, u]`.
In other words, The 1-sublevel set of `V(x, u)` is an invariant subset of `rectangle`
with any state-feedback `κ(x)` such that `V(x, κ(x)) ≤ 1` for any `x` such that
`min_u V(x, u) ≤ 1`.
Such candidate `κ(x)` can therefore be chosen as `argmin_u V(x, u)`.
Let `inv(Q) = U' * U` where `U = [Ux Uu]`. We have `V(x, u) = ||Ux * x + Uu * u||_2`.
`κ(x)` is therefore the least-square solution of `Uu * κ(x) = -Ux * x`.
This we find the linear state-feedback `κ(x) = K * x` where `K = -Uu \ Ux`.

In [10]:
P = inv(Symmetric(value.(Q)))
using LinearAlgebra
F = cholesky(P)
K = -F.U[:, (n_x + 1):(nD)] \ F.U[:, 1:n_x] # That gives the following state feedback in polynomial form:

2×6 Matrix{Float64}:
  1.80448e-15  -0.664213      8.67445e-16  …   5.77456e-15  -8.66686e-16
 -0.0475248    -1.29313e-15  -0.080485        -0.0090073    -0.0181524

The corresponding polynomial form is given by:

In [11]:
k = K * x

2-element Vector{DynamicPolynomials.Polynomial{true, Float64}}:
 1.8044831922515837e-15x₁ - 0.6642133496066044x₂ + 8.674450511927789e-16x₃ - 0.7227866208948744x₄ + 5.77456371634203e-15x₅ - 8.666856822362621e-16x₆
 -0.047524750631232654x₁ - 1.2931340320941731e-15x₂ - 0.08048496745811395x₃ + 2.2751844292478188e-15x₄ - 0.009007296539577787x₅ - 0.01815235310760693x₆

We now have two equivalent ways to obtain the Lyapunov function.
Because `{V(x) ≤ 1} = {min_u V(x, u) ≤ 1}`,
see the left-hand side as the projection of the ellipsoid on `x, u`.
As the projection on the polar becomes simply cutting with the hyperplane `u = 0`,
the polar of the projection is simply `Q[1:6, 1:6]` ! So

In [12]:
Px = inv(Symmetric(value.(Q[1:n_x, 1:n_x])))

6×6 LinearAlgebra.Symmetric{Float64, Matrix{Float64}}:
  0.60379       4.58007e-15   0.0985877    …   0.465159      0.0588733
  4.58007e-15   1.32353       5.85663e-15      1.92455e-15  -3.30081e-15
  0.0985877     5.85663e-15   1.85572          2.64415       0.619223
 -1.63448e-16   0.480075     -5.95808e-15      7.21166e-16  -5.3412e-16
  0.465159      1.92455e-15   2.64415         14.0161        1.84333
  0.0588733    -3.30081e-15   0.619223     …   1.84333       0.933395

An alternative way is to use our linear state feedback.
We know that `min_u V(x, u) = V(x, Kx)` so

In [13]:
Px = [I; K]' * P * [I; K]

6×6 Matrix{Float64}:
  0.60379       4.58007e-15   0.0985877    …   0.465159      0.0588733
  4.58007e-15   1.32353       5.85663e-15      1.92455e-15  -3.30081e-15
  0.0985877     5.85663e-15   1.85572          2.64415       0.619223
 -1.63448e-16   0.480075     -5.95808e-15      7.21166e-16  -5.3412e-16
  0.465159      1.92455e-15   2.64415         14.0161        1.84333
  0.0588733    -3.30081e-15   0.619223     …   1.84333       0.933395

We can double check that this matrix is negative definite:

In [14]:
eigen(Symmetric(Px * (A + B * K) + (A + B * K)' * Px)).values

6-element Vector{Float64}:
 -243.57396135575334
   -0.88551014883334
   -2.072629222450229e-5
   -7.2574026264248e-8
    2.816678689171484e-7
    0.0009642423807258547

Let's now find a valid Lyapunov function for the nonlinear system
using that linear state feedback.
That corresponds to the V-step of [YAP21, Algorithm 1]:

In [15]:
function _create(model, d, P)
    if d isa Int
        return @variable(model, variable_type = P(monomials(x, 0:d)))
    else
        return d
    end
end
function base_model(solver, V, k, s3, γ)
    model = SOSModel(solver)
    V = _create(model, V, Poly)
    k = _create.(model, k, Poly)
    s3 = _create(model, s3, SOSPoly)
    ∂ = differentiate # Let's use a fancy shortcut
    @constraint(model, ∂(V, x) ⋅ (f + g * k) <= s3 * (V - γ)) # [YAP21, (E.2)]
    for r in inequalities(X)
        @constraint(model, V >= γ, domain = @set(r >= 0)) # [YAP21, (E.3)]
    end
    return model, V, k, s3
end

_degree(d::Int) = d
_degree(V) = maxdegree(V)

function V_step(solver, V0, γ, k, s3)
    model, V, k, s3 = base_model(solver, _degree(V0), k, s3, γ)
    if !(V0 isa Int)
        @constraint(model, V >= γ, domain = @set(V0 >= γ)) # [YAP21, (E.6)]
    end
    optimize!(model)
    return model, value(V)
end

γ = 1.0
s3 = 1.0
model, V = V_step(solver, 2, γ, k, s3)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Dual objective value : -7.48410e-12

* Work counters
  Solve time (sec)   : 7.24003e-03


The Lyapunov obtained is as follows

In [16]:
V

4.8877630431071766e-8x₁² + 2.758644385189247e-25x₁x₂ - 2.5017256249934794e-11x₁x₃ - 1.6560856778462455e-22x₁x₄ + 7.2054429011152e-8x₁x₅ + 8.57308533581914e-9x₁x₆ + 1.6477243793941312e-7x₂² - 1.356277820272328e-23x₂x₃ - 9.405327948946013e-8x₂x₄ - 1.5496090022096714e-21x₂x₅ + 4.1228958368328845e-24x₂x₆ + 1.3474255785621875e-11x₃² + 2.1826135093406784e-24x₃x₄ + 7.903343565693963e-12x₃x₅ + 1.1325867942533704e-12x₃x₆ + 1.1845327918988095e-7x₄² + 2.325800791173985e-21x₄x₅ - 7.317927329413892e-23x₄x₆ + 3.6918652669160694e-7x₅² + 9.216771989621136e-9x₅x₆ + 1.1271560584848727e-8x₆² + 2.522857862159588e-21x₁ + 4.0111752884416337e-7x₂ - 2.4611045730186663e-21x₃ - 4.106687671295625e-7x₄ - 2.4896278530919443e-22x₅ - 3.134514575443906e-22x₆ + 1.0000068640913347

We now try to find a state feedback that would improve γ

In [17]:
using MutableArithmetics
function γ_step(solver, V, γ_min, k_best, s3_best, degree_k, degree_s3, γ_tol, max_iters)
    γ0_min = γ_min
    γ_max = Inf
    num_iters = 0
    while γ_max - γ_min > γ_tol && num_iters < max_iters
        if isfinite(γ_max)
            γ = (γ_min + γ_max) / 2
        else
            γ = γ0_min + (γ_min - γ0_min + 1) * 2
        end
        model, V, k, s3 = base_model(solver, V, degree_k, degree_s3, γ)
        num_iters += 1
        @info("Iteration $num_iters/$max_iters : solving...")
        optimize!(model)
        if primal_status(model) == MOI.FEASIBLE_POINT
            γ_min = γ
            k_best = value.(k)
            s3_best = value(s3)
        elseif dual_status(model) == MOI.INFEASIBILITY_CERTIFICATE
            γ_max = γ
        else
            @warn("Giving up $(raw_status(model)), $(termination_status(model)), $(primal_status(model)), $(dual_status(model))")
            break
        end
        @info("Solved in $(solve_time(model)) : γ ∈ ]$γ_min, $γ_max]")
    end
    if !isfinite(γ_max)
        error("Cannot find any infeasible γ")
    end
    return γ_min, k_best, s3_best
end

γ, k, s3 = γ_step(solver, V, γ, k, s3, [2, 2], 2, 1e-3, 10)

[ Info: Iteration 1/10 : solving...
[ Info: Solved in 0.08807748800000001 : γ ∈ ]1.0, 3.0]
[ Info: Iteration 2/10 : solving...
[ Info: Solved in 0.089961522 : γ ∈ ]1.0, 2.0]
[ Info: Iteration 3/10 : solving...
[ Info: Solved in 0.089648016 : γ ∈ ]1.0, 1.5]
[ Info: Iteration 4/10 : solving...
[ Info: Solved in 0.08899781699999999 : γ ∈ ]1.0, 1.25]
[ Info: Iteration 5/10 : solving...
[ Info: Solved in 0.088760728 : γ ∈ ]1.0, 1.125]
[ Info: Iteration 6/10 : solving...
[ Info: Solved in 0.08902463399999999 : γ ∈ ]1.0, 1.0625]
[ Info: Iteration 7/10 : solving...
[ Info: Solved in 0.088709227 : γ ∈ ]1.0, 1.03125]
[ Info: Iteration 8/10 : solving...
[ Info: Solved in 0.08886092999999999 : γ ∈ ]1.0, 1.015625]
[ Info: Iteration 9/10 : solving...
[ Info: Solved in 0.086804593 : γ ∈ ]1.0, 1.0078125]
[ Info: Iteration 10/10 : solving...
[ Info: Solved in 0.08838452200000001 : γ ∈ ]1.0, 1.00390625]


(1.0, DynamicPolynomials.Polynomial{true, Float64}[1.8044831922515837e-15x₁ - 0.6642133496066044x₂ + 8.674450511927789e-16x₃ - 0.7227866208948744x₄ + 5.77456371634203e-15x₅ - 8.666856822362621e-16x₆, -0.047524750631232654x₁ - 1.2931340320941731e-15x₂ - 0.08048496745811395x₃ + 2.2751844292478188e-15x₄ - 0.009007296539577787x₅ - 0.01815235310760693x₆], 1.0)

We now try to find a new Lyapunov V:

In [18]:
model, V = V_step(solver, V, γ, k, s3)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Dual objective value : -7.83208e-12

* Work counters
  Solve time (sec)   : 7.53024e-03


The Lyapunov obtained is as follows

In [19]:
V

4.376042942500143e-8x₁² + 3.0854962061294627e-23x₁x₂ - 1.7601002249639244e-11x₁x₃ - 2.765030844114297e-22x₁x₄ + 6.256328709552058e-8x₁x₅ + 8.163335173800878e-9x₁x₆ + 1.5675824258621558e-7x₂² - 6.181375313200897e-24x₂x₃ - 8.953282509687231e-8x₂x₄ - 1.4165786657923989e-21x₂x₅ - 7.175905734768593e-24x₂x₆ + 1.0291543148057179e-11x₃² + 2.305093354599287e-24x₃x₄ + 3.5279734729918183e-12x₃x₅ + 4.047069941707634e-13x₃x₆ + 1.1177983874209226e-7x₄² + 1.8997021610332166e-21x₄x₅ - 6.609225591329223e-23x₄x₆ + 3.283751981760004e-7x₅² + 8.9573678236505e-9x₅x₆ + 1.0119892160453752e-8x₆² + 6.992970497821096e-22x₁ + 3.742741602444346e-7x₂ + 2.1833265337731446e-21x₃ - 3.8905178334301527e-7x₄ - 5.73704135856962e-22x₅ - 2.3086032179199843e-22x₆ + 1.000006491976757

We now try to improve γ again

In [20]:
γ, k, s3 = γ_step(solver, V, γ, k, s3, [2, 2], 2, 1e-3, 10)

[ Info: Iteration 1/10 : solving...
[ Info: Solved in 0.086352785 : γ ∈ ]1.0, 3.0]
[ Info: Iteration 2/10 : solving...
[ Info: Solved in 0.086826993 : γ ∈ ]1.0, 2.0]
[ Info: Iteration 3/10 : solving...
[ Info: Solved in 0.086824394 : γ ∈ ]1.0, 1.5]
[ Info: Iteration 4/10 : solving...
[ Info: Solved in 0.086658189 : γ ∈ ]1.0, 1.25]
[ Info: Iteration 5/10 : solving...
[ Info: Solved in 0.08705609800000001 : γ ∈ ]1.0, 1.125]
[ Info: Iteration 6/10 : solving...
[ Info: Solved in 0.086787092 : γ ∈ ]1.0, 1.0625]
[ Info: Iteration 7/10 : solving...
[ Info: Solved in 0.087680908 : γ ∈ ]1.0, 1.03125]
[ Info: Iteration 8/10 : solving...
[ Info: Solved in 0.087091998 : γ ∈ ]1.0, 1.015625]
[ Info: Iteration 9/10 : solving...
[ Info: Solved in 0.086815793 : γ ∈ ]1.0, 1.0078125]
[ Info: Iteration 10/10 : solving...
[ Info: Solved in 0.088640327 : γ ∈ ]1.0, 1.00390625]


(1.0, DynamicPolynomials.Polynomial{true, Float64}[1.8044831922515837e-15x₁ - 0.6642133496066044x₂ + 8.674450511927789e-16x₃ - 0.7227866208948744x₄ + 5.77456371634203e-15x₅ - 8.666856822362621e-16x₆, -0.047524750631232654x₁ - 1.2931340320941731e-15x₂ - 0.08048496745811395x₃ + 2.2751844292478188e-15x₄ - 0.009007296539577787x₅ - 0.01815235310760693x₆], 1.0)

We now try to find a new Lyapunov V:

In [21]:
model, V = V_step(solver, V, γ, k, s3)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Dual objective value : -7.83164e-12

* Work counters
  Solve time (sec)   : 7.50414e-03


The Lyapunov obtained is as follows

In [22]:
V

4.37604291697808e-8x₁² - 2.5010386698695297e-22x₁x₂ - 1.760100189200634e-11x₁x₃ + 2.4002793516793827e-22x₁x₄ + 6.256328698140404e-8x₁x₅ + 8.163335155209215e-9x₁x₆ + 1.5675824207942018e-7x₂² + 1.2680737524663406e-23x₂x₃ - 8.953282489562196e-8x₂x₄ - 1.2248358346399777e-21x₂x₅ + 3.649234736682948e-23x₂x₆ + 1.0291543003783761e-11x₃² + 1.2485592561346202e-24x₃x₄ + 3.5279734615338094e-12x₃x₅ + 4.0470698639118724e-13x₃x₆ + 1.1177983853360767e-7x₄² + 2.5700330230270258e-21x₄x₅ - 6.889623187187117e-23x₄x₆ + 3.283751980548072e-7x₅² + 8.957367813629968e-9x₅x₆ + 1.0119892155969199e-8x₆² + 2.6130508726689894e-22x₁ + 3.7427416037934315e-7x₂ - 2.634580144814241e-21x₃ - 3.8905178257878557e-7x₄ - 1.877416351720248e-21x₅ - 2.821008879571607e-22x₆ + 1.0000064919767413

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*